In [30]:
import hdf5_getters 
import hdf5_descriptors as dsp
import hdf5_utils as utils
import h5py
import pandas as pd
import numpy as np
import glob
import os

In [31]:
def apply_to_all_files(basedir, func=lambda x: x, ext='h5'):
    """
    Go through all subdirectories starting from base directory
    Apply func to all the files 
    """
    count = 0
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root, '*'+ext))
        for f in files : 
            func(f, count)
            count +=1
        

In [32]:
#get the column names
col = []
# get all getters
getters = filter(lambda x: x[:4] == 'get_' and x[4:]!='num_songs', hdf5_getters.__dict__.keys())

# get the feature names
for getter in getters:
        col.append(getter[4:])   
        
print(col)

['artist_familiarity', 'artist_hotttnesss', 'artist_id', 'artist_mbid', 'artist_playmeid', 'artist_7digitalid', 'artist_latitude', 'artist_longitude', 'artist_location', 'artist_name', 'release', 'release_7digitalid', 'song_id', 'song_hotttnesss', 'title', 'track_7digitalid', 'similar_artists', 'artist_terms', 'artist_terms_freq', 'artist_terms_weight', 'analysis_sample_rate', 'audio_md5', 'danceability', 'duration', 'end_of_fade_in', 'energy', 'key', 'key_confidence', 'loudness', 'mode', 'mode_confidence', 'start_of_fade_out', 'tempo', 'time_signature', 'time_signature_confidence', 'track_id', 'segments_start', 'segments_confidence', 'segments_pitches', 'segments_timbre', 'segments_loudness_max', 'segments_loudness_max_time', 'segments_loudness_start', 'sections_start', 'sections_confidence', 'beats_start', 'beats_confidence', 'bars_start', 'bars_confidence', 'tatums_start', 'tatums_confidence', 'artist_mbtags', 'artist_mbtags_count', 'year']


In [33]:
song_db = pd.DataFrame(columns=col);

def get_all_data(filename, index):
    individual_row = []
    h5=hdf5_getters.open_h5_file_read(filename)
    getters = filter(lambda x: x[:4] == 'get_' and x[4:]!='num_songs', hdf5_getters.__dict__.keys())

    # get the feature values
    for getter in getters:
        res = hdf5_getters.__getattribute__(getter)(h5)
        if(getter == 'get_artist_terms' or getter == 'get_similar_artists'):
            res = res.astype(str) 
        individual_row.append(res);
    song_db.loc[index] = individual_row
    h5.close()

In [35]:
print(song_db.artist_mbtags)

Series([], Name: artist_mbtags, dtype: object)


In [28]:
database_path = './MillionSongSubset/data/'
apply_to_all_files(database_path, func = get_all_data)

In [29]:
song_db.to_csv('MillionSongDB.csv', index=False)